# Your Title Here

**Name(s)**: Ethan Cao

**Website Link**: (your website link)

## Code

In [7]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Framing the Problem

In [132]:
def combine_times(date_col_name, time_col_name, new_col_name, df):
    df = df.copy()
    df[new_col_name] = df[date_col_name] + pd.to_timedelta(df[time_col_name].astype(str))
    return df

data = pd.read_excel("outage.xlsx", skiprows=[0,1,2,3,4,6], index_col=1).iloc[:,1:]
data = combine_times("OUTAGE.START.DATE", 'OUTAGE.START.TIME', 'OUTAGE.START.DATETIME', data)
data = combine_times("OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME", "OUTAGE.RESTORATION.DATETIME", data)

In [133]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaN,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


### Baseline Model

In [235]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

# Load the dataset
# Remove rows with missing target variable (OUTAGE.DURATION)
# data = data.dropna(subset=['OUTAGE.DURATION'])

col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE'])
    ], remainder='passthrough')

pl = Pipeline([
    ('col-trans', col_trans),
    ('lin-reg', LinearRegression())
])


# Select relevant features and target variable
selected_features_and_target = data[['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE','POSTAL.CODE','OUTAGE.DURATION']].dropna()
X = selected_features_and_target.drop(columns=['OUTAGE.DURATION'])
y = selected_features_and_target['OUTAGE.DURATION']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Model Training without Feature Engineering
pl.fit(X_train, y_train)

# Predictions on the test set
y_pred = pl.predict(X_test)

In [236]:
pl.score(X_test, y_test)

-0.0218092395423084

In [237]:
np.sqrt(mean_squared_error(y_test, y_pred))

7309.932857130835

In [238]:
import pprint as pp

### Final Model

In [239]:
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE']),
    ('num-features', make_pipeline(StandardScaler(),PolynomialFeatures()), ['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE'])
    ], remainder='passthrough')

# Model Training with Feature Engineering using Pipelines and GridSearchCV
model = make_pipeline(
    col_trans,
    LinearRegression()
)

param_grid = {
    'columntransformer__num-features__polynomialfeatures__degree': [2,3,4,5]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Cross-Validation):", np.sqrt(-grid_search.best_score_))

Best Parameters: {'columntransformer__num-features__polynomialfeatures__degree': 2}
Best Score (Cross-Validation): 5917.57203567354


In [247]:
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE']),
    ('num-features', make_pipeline(StandardScaler(),PolynomialFeatures(degree=2)), ['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE'])
    ], remainder='passthrough')

# Model Training with Feature Engineering using Pipelines and GridSearchCV
model = make_pipeline(
    col_trans,
    LinearRegression()
)
scores = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    scores.append(model.score(X_test, y_test))

np.mean(scores)

-794848.6846175174

In [251]:
scores.sort()
scores

[-722179525.7501012,
 -72669046.84747443,
 -3.9653751355864886,
 -3.454158537152196,
 -3.2921759309188277,
 -3.221185930179968,
 -3.2138208656244416,
 -3.115855637729009,
 -2.5431380853077865,
 -2.5243598265105356,
 -2.4764471391338905,
 -2.4651183870071436,
 -2.4377759190038164,
 -2.4075353052367428,
 -2.3923926692808646,
 -2.3444378379054016,
 -2.2035233645747767,
 -2.1938227585381305,
 -2.1537904359415396,
 -2.1045061569518606,
 -2.0163475529514656,
 -2.00068797176746,
 -1.9943509628632716,
 -1.9630167060343457,
 -1.924002316128099,
 -1.8838542132195033,
 -1.8536482300036243,
 -1.753486659081196,
 -1.7408492695879114,
 -1.6803033421651343,
 -1.5732984328004238,
 -1.570498731339364,
 -1.5497841555520906,
 -1.5183610264428458,
 -1.5130602697977413,
 -1.4985588191624784,
 -1.4666146149691204,
 -1.4246045514791463,
 -1.39703194783521,
 -1.354808735000423,
 -1.3256263777867874,
 -1.2989262770135563,
 -1.2851146104535265,
 -1.2849557739939508,
 -1.28072127086712,
 -1.259909490997229,
 -1.

### Fairness Analysis

In [ ]:
# TODO